# Operations Research Take home assignment

I have tried to solve the problem using the pyomo framework and glpk solver. I have not incorporated crossdocking functionality into the solution. 

### IMPORTING THE ENVIRONMENTS

In [27]:
import pandas as pd
from pyomo.environ import Set, SolverFactory, ConcreteModel, Constraint, Param, Objective, Var, Binary, minimize
from datetime import datetime, timedelta, time
import numpy as np

### LOADING THE DATA

In [28]:
shipments = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_shipments')
df = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_schedule')
capacity = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_capacity')

### Schedule data processing 
The departure times are a list of tuples but stored as string. My goal is to parse them out as tuples, then explode the dataset to find all the possible linehaul routes, and then to parse the datetime fields. 

In [29]:
def convert_string_to_list(s):
    # Replace colons with quotes to make it a valid Python expression
    s = s.replace(", ", ",").replace("[","").replace("]","")
    return s

df['new_departure']=df['scheduled_depart_weekday_time_local'].apply(convert_string_to_list)

In [30]:
def parse_new_departure(s):
    tuples = s.split('),(')
    tuples = [t.replace('(', '').replace(')', '') for t in tuples]
    parsed_tuples = [tuple(t.split(',')) for t in tuples]
    return parsed_tuples

In [31]:
# Apply the function to the new_departure column
df['parsed_departures'] = df['new_departure'].apply(parse_new_departure)

# Explode the parsed_departures column
schedule = df.explode('parsed_departures')

# Create the day_of_week and departure_time columns
schedule[['day_of_week', 'departure_time']] = pd.DataFrame(schedule['parsed_departures'].tolist(), index=schedule.index)

# Convert the day_of_week to integer
schedule['day_of_week'] = schedule['day_of_week'].astype(int)

# Drop the parsed_departures column
schedule['parsed_departure_time'] = schedule['departure_time'].apply(lambda x: datetime.strptime(x, "%H:%M").time())

# Reset index if necessary
schedule.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
schedule.head(5)

,carrier_id,origin_region,destination_region,scheduled_depart_weekday_time_local,travel_hrs_with_timezone_offset,truck_type,rate_type,rate_break,new_departure,parsed_departures,day_of_week,departure_time,parsed_departure_time
0,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(1, 20:00)",1,20:00,20:00:00
1,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(2, 20:00)",2,20:00,20:00:00
2,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(3, 20:00)",3,20:00,20:00:00
3,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(4, 20:00)",4,20:00,20:00:00
4,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(5, 20:00)",5,20:00,20:00:00


### Combining datetime schedule with the date range of scheduled shipments 

I find the min and max dates in the scheduled shipments, and with some buffer time (-5 days) join them onto the linehaul schedule

In [32]:
max_dt=max(shipments['due_time_local'])
min_dt=min(shipments['ready_time_local'])-timedelta(days=5)
m=pd.date_range(min_dt, max_dt, freq='D')
df_dates = pd.DataFrame(m, columns=['date'])
df_dates['date_only']=df_dates['date'].dt.date
df_dates['dow'] = df_dates['date'].dt.dayofweek+1
df_dates

,date,date_only,dow
0,2024-05-04 10:00:00,2024-05-04,6
1,2024-05-05 10:00:00,2024-05-05,7
2,2024-05-06 10:00:00,2024-05-06,1
3,2024-05-07 10:00:00,2024-05-07,2
4,2024-05-08 10:00:00,2024-05-08,3
5,2024-05-09 10:00:00,2024-05-09,4
6,2024-05-10 10:00:00,2024-05-10,5
7,2024-05-11 10:00:00,2024-05-11,6
8,2024-05-12 10:00:00,2024-05-12,7
9,2024-05-13 10:00:00,2024-05-13,1


In [33]:
schedule=schedule.merge(df_dates, how='left', left_on='day_of_week', right_on='dow')
schedule['departure_dt_parsed']=pd.to_datetime(schedule['date_only'].astype(str) + ' ' + schedule['departure_time'])

### Optimization model

I use pyomo to build an optimization model with the following steps:
1. define sets - shipments and linehauls 
2. define the shipment parameters - weights, pallets, ready and due times 
3. define the linehaul parameters - departure times
4. define the objective function - minimize the total cost of the shipments and balance with on-time performance. I use the alpha parameter to balance the two, which can be varied based on business objective. 
5. define the constraints for pallets and weights 
6. Since I am only solving for FTL and linehaul, I also apply a constraint that the loads can only be transported using one scheduled FTL linehaul 
7. I have also implemented a constraint for shipments to arrive at or after their due time

In [34]:
# Initialize the model
model = ConcreteModel()

# Define sets
model.SHIPMENTS = Set(initialize=shipments['shipment_id'].tolist())
model.LINEHAULS = Set(initialize=schedule.index.tolist())

In [35]:
# Parameters
shipment_weight = dict(zip(shipments['shipment_id'], shipments['weight_kg']))
shipment_pallet = dict(zip(shipments['shipment_id'], shipments['pallet']))
shipment_ready_time = dict(zip(shipments['shipment_id'], pd.to_datetime(shipments['ready_time_local'])))
shipment_due_time = dict(zip(shipments['shipment_id'], pd.to_datetime(shipments['due_time_local'])))

#shipment_ready_time = dict(zip(shipments['shipment_id'], (pd.to_datetime(shipments['ready_time_local']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')))
#shipment_due_time = dict(zip(shipments['shipment_id'], (pd.to_datetime(shipments['due_time_local']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')))


In [36]:
linehaul_departure_times = dict(zip(schedule.index, pd.to_datetime(schedule['departure_dt_parsed'])))
linehaul_departure_times

{0: Timestamp('2024-05-06 20:00:00'),
 1: Timestamp('2024-05-13 20:00:00'),
 2: Timestamp('2024-05-07 20:00:00'),
 3: Timestamp('2024-05-14 20:00:00'),
 4: Timestamp('2024-05-08 20:00:00'),
 5: Timestamp('2024-05-15 20:00:00'),
 6: Timestamp('2024-05-09 20:00:00'),
 7: Timestamp('2024-05-16 20:00:00'),
 8: Timestamp('2024-05-10 20:00:00'),
 9: Timestamp('2024-05-17 20:00:00'),
 10: Timestamp('2024-05-06 20:00:00'),
 11: Timestamp('2024-05-13 20:00:00'),
 12: Timestamp('2024-05-07 20:00:00'),
 13: Timestamp('2024-05-14 20:00:00'),
 14: Timestamp('2024-05-08 20:00:00'),
 15: Timestamp('2024-05-15 20:00:00'),
 16: Timestamp('2024-05-09 20:00:00'),
 17: Timestamp('2024-05-16 20:00:00'),
 18: Timestamp('2024-05-10 20:00:00'),
 19: Timestamp('2024-05-17 20:00:00'),
 20: Timestamp('2024-05-06 12:00:00'),
 21: Timestamp('2024-05-13 12:00:00'),
 22: Timestamp('2024-05-07 12:00:00'),
 23: Timestamp('2024-05-14 12:00:00'),
 24: Timestamp('2024-05-08 12:00:00'),
 25: Timestamp('2024-05-15 12:00:00

### LTL costs
I wrote a function to parse the LTL costs in case it is used

In [37]:
# Function to parse rate breaks
def parse_rate_break(rate_break):
    if isinstance(rate_break, str):
        rate_break = eval(rate_break)
    if isinstance(rate_break, list):
        rate_break = rate_break[0]
    return rate_break

schedule['parsed_rate_break'] = schedule['rate_break'].apply(parse_rate_break)

Defining the parameters

In [38]:
# Parameters in the model
model.weight = Param(model.SHIPMENTS, initialize=shipment_weight)
model.pallet = Param(model.SHIPMENTS, initialize=shipment_pallet)

model.ready_time_og = dict(zip(shipments['shipment_id'], pd.to_datetime(shipments['ready_time_local'])))
model.ready_time = Param(model.SHIPMENTS, initialize={k: (v - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') for k, v in shipment_ready_time.items()})
#model.ready_time = Param(model.SHIPMENTS, initialize=shipment_ready_time)


model.due_time_og = Param(model.SHIPMENTS, initialize=shipment_due_time)
model.due_time = Param(model.SHIPMENTS, initialize={k: (v - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') for k, v in shipment_due_time.items()})
#model.due_time = Param(model.SHIPMENTS, initialize=shipment_due_time)

model.travel_time = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['travel_hrs_with_timezone_offset'])))

model.departure_time_og = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['departure_dt_parsed'])))
#model.departure_time = Param(model.LINEHAULS, initialize={k: (v - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') for k, v in linehaul_departure_times.items()})
model.departure_time = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, (schedule['departure_dt_parsed'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))))


Defining the model variables and the trade off parameters

In [39]:
# Variables
model.x = Var(model.SHIPMENTS, model.LINEHAULS, domain=Binary)

In [40]:
# Trade-off parameter (0 <= alpha <= 1)
alpha = 0.5  # You can adjust this value to balance between cost and OTP

Defining a function to calculate LTL costs to be used in the objective function

In [41]:
def calculate_ltl_cost(weight, rate_break):
    applicable_rate = max(rate_break.keys())
    for threshold, rate in sorted(rate_break.items()):
        if weight <= threshold:
            applicable_rate = rate
            break
    return weight * applicable_rate

The objective function

In [42]:
# Objective function: Minimize weighted sum of cost and OTP
def obj_rule(model):
    cost_term = sum(model.x[s, l] * (model.weight[s] / capacity.loc[0, 'max_weight_kgs']) * schedule['parsed_rate_break'][l]
                    for s in model.SHIPMENTS for l in model.LINEHAULS if schedule.loc[l, 'rate_type'] == 'FTL') + \
                sum(model.x[s, l] * calculate_ltl_cost(model.weight[s], schedule['parsed_rate_break'][l])
                    for s in model.SHIPMENTS for l in model.LINEHAULS if schedule.loc[l, 'rate_type'] == 'LTL')
    
    otp_term = sum(model.x[s, l] * (model.due_time[s] - (model.ready_time[s] + model.travel_time[l] * 3600))
                   for s in model.SHIPMENTS for l in model.LINEHAULS if (model.due_time[s] - (model.ready_time[s] + model.travel_time[l] * 3600)) >= 0)
    
    return (1 - alpha) * cost_term - alpha * otp_term

model.obj = Objective(rule=obj_rule, sense=minimize)

In [43]:
# Constraints
def route_constraint(model, s):
    return sum(model.x[s, l] for l in model.LINEHAULS) <=3
model.route_con = Constraint(model.SHIPMENTS, rule=route_constraint)

In [44]:
def weight_constraint(model, l):
    return sum(model.weight[s] * model.x[s, l] for s in model.SHIPMENTS) <= capacity.loc[0, 'max_weight_kgs']

model.weight_con = Constraint(model.LINEHAULS, rule=weight_constraint)

In [45]:
def pallet_constraint(model, l):
    return sum(model.pallet[s] * model.x[s, l] for s in model.SHIPMENTS) <= capacity.loc[0, 'max_pallet_count']

model.pallet_con = Constraint(model.LINEHAULS, rule=pallet_constraint)

In [46]:
# # Due time constraint
# def due_time_constraint(model, s):
#     #print( model.due_time[s])
#     #print(sum(model.ready_time[s] + model.travel_time[l] * 3600 * model.x[s, l] for l in model.LINEHAULS))
#     #print(model.due_time[s] >= sum(model.ready_time[s] + model.travel_time[l] * 3600 * model.x[s, l] for l in model.LINEHAULS))
    
#     #return model.due_time[s] >= sum(model.ready_time[s] + model.travel_time[l] * 3600 * model.x[s, l] for l in model.LINEHAULS)
#     return model.due_time[s] >= sum((model.departure_time[l] + model.travel_time[l] * 3600) * model.x[s, l] for l in model.LINEHAULS)

# model.due_time_con = Constraint(model.SHIPMENTS, rule=due_time_constraint)

In [47]:
# #Departure time constraint
# def departure_time_constraint(model, s, l):
#     ready_time = model.ready_time_og[s]
#     departure_dt=model.departure_time_og[l]
#     print(ready_time)
#     print(departure_dt)
#     print(departure_dt >= ready_time)
#     # Calculate the departure datetime
#     #ready_time_dt = pd.to_datetime(ready_time, unit='s')
#     #departure_datetime = datetime.combine(ready_time_dt.date(), departure_time) + timedelta(days=(int(departure_day - ready_time_dt.weekday() - 1)) % 7)
#     if departure_dt >= ready_time:
#         return Constraint.Feasible
#     else:
#         return Constraint.Infeasible
# model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)

In [48]:
# Departure time constraint
# def departure_time_constraint(model, s, l):
#     ready_time = model.ready_time[s]
#     departure_day = schedule.loc[l, 'day_of_week']
#     departure_time = schedule.loc[l, 'parsed_departure_time']
    
#     # Calculate the departure datetime
#     ready_time_dt = pd.to_datetime(ready_time, unit='s')
#     departure_datetime = datetime.combine(ready_time_dt.date(), departure_time) + timedelta(days=int((departure_day - ready_time_dt.weekday() - 1)) % 7)
    
#     if departure_datetime.timestamp() >= ready_time:
#         return Constraint.Feasible
#     return Constraint.Infeasible

# model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)


In [49]:
# Departure time constraint
# def departure_time_constraint(model, s, l):
#     if model.departure_time_og[l].timestamp() >= model.ready_time[s]:
#         return Constraint.Feasible
#     else:
#         return Constraint.Infeasible
# model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)


In [50]:
# test=[]
# for i in shipment_ready_time:
#     for j in linehaul_departure_times:
#         a=shipment_ready_time[i]<linehaul_departure_times[j].timestamp()
#         if a==False:
#             test.append([a, i, j])

# test

In [51]:
# # Departure time constraint
# def departure_time_constraint(model, s, l):
#     #departure_time=schedule['departure_dt_parsed']
#     if model.departure_time[l] >= model.ready_time[s]:
#         return Constraint.Feasible
#     else:
#         return Constraint.Infeasible
# model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)


In [52]:
# #Departure time constraint
# def departure_time_constraint(model, s, l):
#    return model.departure_time[l] >= model.ready_time[s]
# model.departure_time_con2 = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)

In [53]:
# Departure time constraint
def departure_time_constraint(model, s, l):
    return model.departure_time[l] >= model.ready_time[s] * model.x[s, l]

model.departure_time_con4 = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)

In [54]:
linehaul_departure_times

{0: Timestamp('2024-05-06 20:00:00'),
 1: Timestamp('2024-05-13 20:00:00'),
 2: Timestamp('2024-05-07 20:00:00'),
 3: Timestamp('2024-05-14 20:00:00'),
 4: Timestamp('2024-05-08 20:00:00'),
 5: Timestamp('2024-05-15 20:00:00'),
 6: Timestamp('2024-05-09 20:00:00'),
 7: Timestamp('2024-05-16 20:00:00'),
 8: Timestamp('2024-05-10 20:00:00'),
 9: Timestamp('2024-05-17 20:00:00'),
 10: Timestamp('2024-05-06 20:00:00'),
 11: Timestamp('2024-05-13 20:00:00'),
 12: Timestamp('2024-05-07 20:00:00'),
 13: Timestamp('2024-05-14 20:00:00'),
 14: Timestamp('2024-05-08 20:00:00'),
 15: Timestamp('2024-05-15 20:00:00'),
 16: Timestamp('2024-05-09 20:00:00'),
 17: Timestamp('2024-05-16 20:00:00'),
 18: Timestamp('2024-05-10 20:00:00'),
 19: Timestamp('2024-05-17 20:00:00'),
 20: Timestamp('2024-05-06 12:00:00'),
 21: Timestamp('2024-05-13 12:00:00'),
 22: Timestamp('2024-05-07 12:00:00'),
 23: Timestamp('2024-05-14 12:00:00'),
 24: Timestamp('2024-05-08 12:00:00'),
 25: Timestamp('2024-05-15 12:00:00

In [55]:
# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)
print(results)

In [ ]:
# Output
output = []
for s in model.SHIPMENTS:
    for l in model.LINEHAULS:
          if model.x[s, l].value > 0:
            departure_datetime = schedule.loc[l, 'departure_dt_parsed']
            drop_datetime = departure_datetime + timedelta(hours=int(schedule.loc[l, 'travel_hrs_with_timezone_offset']))
            rate_type = schedule.loc[l, 'rate_type']
            
            truck_id = f"{schedule.loc[l, 'origin_region']}-{schedule.loc[l, 'destination_region']}-{rate_type}-{departure_datetime.strftime('%Y%m%d')}-0"
            share_of_cost_based_on_weight = (model.weight[s] / capacity.loc[0, 'max_weight_kgs']) * schedule['parsed_rate_break'][l] if rate_type == 'FTL' else calculate_ltl_cost(model.weight[s], schedule['parsed_rate_break'][l])
            
            output.append({
                'shipment_id': s,
                'leg_id': l,
                'origin_region': shipments.loc[shipments['shipment_id'] == s, 'origin_region'].values[0],
                'destination_region': schedule.loc[l, 'destination_region'],
                'depart_time_local': departure_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                'drop_time_local': drop_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                'carrier_id': schedule.loc[l, 'carrier_id'],
                'truck_id': truck_id,
                'share_of_cost_based_on_weight': share_of_cost_based_on_weight,
                'met_otp': drop_datetime.timestamp() <= model.due_time[s]
            })

output_df = pd.DataFrame(output)


In [ ]:
output_df

## Model Improvements include 
1. Verifying the constraints and the objective function. 
2. Solving the problem to include crossdocking 
3. Adding in some approximations based on business knowledge. examples - max # of crossdocks based on distance,  max allowable splitting of shipments 